# 1. Import and Pre-processing

In [1]:
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn.preprocessing import MinMaxScaler
from scipy.stats import hmean

import cornac
from cornac.utils import cache
from cornac.metrics import NCRR, NDCG, FMeasure
from cornac.eval_methods import BaseMethod
from cornac.models import MostPop, BPR, WMF, VAECF, SVD
from cornac.hyperopt import Discrete, Continuous, RandomSearch

from recommenders.evaluation.python_evaluation import serendipity, distributional_coverage, catalog_coverage

from harmonic_mean import HarmonicMean
from serendipity_wrapper import Serendipity
from combined_eval_method import CombinedBaseMethod
from new_random_search import NewRandomSearch

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


# 2. Helper Functions

In [2]:
# helper function to preprocess without combinine features in train set
def preprocess(dfs):
    for i in range(len(dfs)):

        dfs[i].drop(['Unnamed: 0', 'id', 'dwell_time_ms', 'click', 'created_at', 'updated_at'], axis=1, inplace=True)
        dfs[i] = dfs[i][['userID', 'catID', 'like']]
    
    # reassigning train, test, val
    train, test, val = dfs[0], dfs[1], dfs[2]
    
    return train, test, val

In [3]:
# helper function to preprocess and combine features in train set
def combine_features(dfs_list, like_weight, click_weight, dwell_weight):
    
    # dfs list in [train, test, val] format
    
    dfs = dfs_list
    L = like_weight
    C = click_weight
    D = dwell_weight
    
    for i in range(len(dfs)):
    
        # convert True/False to 1/0 for all dfs
        dfs[i]['like'] = dfs[i]['like'].apply(lambda x: 1 if x else 0)


        # for train set
        if i == 0:
            dfs[i]['click'] = dfs[i]['click'].apply(lambda x: 1 if x else 0)
            dfs[i].drop(['Unnamed: 0', 'id','created_at', 'updated_at'], axis=1, inplace=True)
            dfs[i] = dfs[i][['userID', 'catID', 'like', 'dwell_time_ms', 'click']]


        # for test and val sets
        elif i == 1 or i == 2:
            dfs[i].drop(['Unnamed: 0', 'id', 'dwell_time_ms', 'click', 'created_at', 'updated_at'], axis=1, inplace=True)
            dfs[i] = dfs[i][['userID', 'catID', 'like']]
    
    # reassigning train, test, val
    train, test, val = dfs[0], dfs[1], dfs[2]
    
    # log transform dwell_time
    train['log_dwell_time'] = train['dwell_time_ms'].apply(lambda x: np.log(x))
    train.drop(['dwell_time_ms'], axis=1, inplace=True)
    
    # initialize MinMaxScaler
    scaler = MinMaxScaler()
    
    # normalizing log_dwell_time
    train['norm_log_dwell_time'] = scaler.fit_transform(train[['log_dwell_time']])
    train.drop(['log_dwell_time'], axis=1, inplace=True)
    
    # assigning weights to features
    train_weighted = train.copy()
    train_weighted['rating'] = L*train_weighted['like'] + C*train_weighted['click'] + D* train_weighted['norm_log_dwell_time']
    train_weighted.drop(['like','click','norm_log_dwell_time'], axis=1, inplace=True)
    
    return train_weighted, test, val

In [4]:
# helper function to form data tuples
def form_tuples(df):
    return [tuple(df.iloc[i]) for i in range(len(df))]        

In [5]:
# helper function to generate output
def make_recommendations(MODEL, TRAINING_SET):
    item_id2idx = MODEL.train_set.iid_map
    item_idx2id = list(MODEL.train_set.item_ids)
    user_idx2id = MODEL.train_set.uid_map
    user_idx2id = list(MODEL.train_set.user_ids)
   
    num_users = len(np.unique(user_idx2id))
    
    # For each user, get the list of items that they have rated in train and probe
    rated = TRAINING_SET.groupby('userID')['catID'].agg(lambda x: list(x))
    rated = rated.to_dict()

    rec_result = {}

    for UIDX in range(0, num_users):
        recommendations, scores = MODEL.rank(UIDX)
        rec_result[user_idx2id[UIDX]] = [item_idx2id[i] for i in recommendations]

    # sort results
    rec_result = {key:value for key, value in sorted(rec_result.items(), key=lambda item: item[0])}

    # remove the rated items from rec_results
    for user in rec_result:
        tmp = [x for x in rec_result[user] if x not in rated[user]]
        rec_result[user] = tmp

    return rated, rec_result

# helper function for generating serendipity, distributional coverage and harmonic mean scores
def evaluate(EXPERIMENT, TRAINING_SET):
    
    MODEL = EXPERIMENT.models[0]
    
    # Get disctionary of rated and recommended items
    rated, recommendations = make_recommendations(MODEL, TRAINING_SET)


    ### PREPARING DATAFRAMES ###
    # Make recommended items into a dataframe for MS recommenders
    data = []
    
    # Iterate over the dictionary keys and values
    for key, values in recommendations.items():
        # Iterate over the values and append rows to the data list
        for value in values:
            data.append([key, value])
    
    # Create the DataFrame with the specified columns
    recommendations_df = pd.DataFrame(data, columns=['userID', 'itemID'])

    # Make the testing set into a dataframe for MS recommenders
    train_df = TRAINING_SET[['userID','catID']].reset_index()
    train_df = train_df[['userID','catID']]
    train_df = train_df.rename(columns={'userID':'userID', 'catID':'itemID'})


    ### METRICS ###
    # Calculate serendipity
    serendipity_score = serendipity(train_df, recommendations_df)

    # Calculate coverage
    dist_coverage_score = distributional_coverage(train_df, recommendations_df)

    # Extract 
    for result in EXPERIMENT.result:
        model_name = result.model_name
        fone10 = result.metric_avg_results['F1@10']
        ncrr = result.metric_avg_results['NCRR@-1']
        ndcg = result.metric_avg_results['NDCG@-1']

    # Calculate harmonic mean
    h_mean = hmean([fone10, ncrr, ndcg, dist_coverage_score, serendipity_score])

    print(model_name)
    print(f"F1@10: {fone10:.3f}")
    print(f"NCRR: {ncrr:.3f}")
    print(f"NDCG: {ndcg:.3f}")
    print(f"Distributional coverage: {dist_coverage_score:.3f}")
    print(f"Serendipity: {serendipity_score:.3f}")
    print(f"Harmonic mean: {h_mean:.3f}")
    print()

# 3. Train Function

In [6]:
# function to train model
def train_model(model_name, weighted, split):
    
    # model_name ['WMF', 'BPR', 'VAECF']
    # weighted ['Yes', 'No']
    
    # check if model is available 
    models = ['MostPop', 'WMF', 'BPR', 'VAECF']

    if model_name not in models:
        print('Model not in available models.')
        return   

    # load train test val datasets based on split type
    train = pd.read_csv('model_data/'+split+'_train.csv')
    test = pd.read_csv('model_data/'+split+'_test.csv')
    val = pd.read_csv('model_data/'+split+'_validation.csv')

    # main data dataset
    user = pd.read_csv('Data/user.csv')
    cat = pd.read_csv('Data/cat.csv')
    
    # list of dfs
    dfs = [train, test, val]
    
    # preprocess based on choice if weighted or not
    if weighted == 'Yes':
        # weights for like, click, dwell_time: 0.5, 0.25, 0.25
        train, test, val = combine_features(dfs, 0.5, 0.25, 0.25)
    
    elif weighted == 'No':
        train, test, val = preprocess(dfs)
    
    else:
        print("Enter 'Yes' or 'No' for weighted ratings")
        return   
        
    # forming data tuples
    train_data = form_tuples(train)
    val_data = form_tuples(val)
    test_data = form_tuples(test)

    
    # define base method
    base_method = CombinedBaseMethod.from_splits(
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        verbose=True,
        exclude_unknowns=True,
        )
        
    # define evaluation metrics
    eval_metrics = [
        HarmonicMean(
            10,
            Serendipity(),
            cornac.metrics.FMeasure(k=10),
            cornac.metrics.NCRR(),
            cornac.metrics.NDCG()
        ),
        Serendipity(),
        cornac.metrics.FMeasure(k=10),
        cornac.metrics.NCRR(),
        cornac.metrics.NDCG()
    ]
    
    # define verbose and seed
    VERBOSE = False
    SEED = 2023
    
    if model_name == 'MostPop':
        
        rs_model = MostPop() # not rs
    
    elif model_name == 'WMF':
        
        # define model
        model = WMF(max_iter=200, a=1.0, verbose=VERBOSE, seed=SEED)
        
        # define parameters to perform RandomSearch on
        wmf_params = [
        Discrete('k', list(range(5,101))),
        Continuous('b', low=0.01, high=0.1),
        Continuous('lambda_u', low=0.01, high=0.1),
        Continuous('lambda_v', low=0.01, high=0.1),
        Continuous('learning_rate', low=0.001, high=0.01)
        ]

        rs_model = NewRandomSearch(
            model=model,
            space=wmf_params,
            metric=HarmonicMean(
                10,
                Serendipity(),
                cornac.metrics.FMeasure(k=10),
                cornac.metrics.NCRR(),
                cornac.metrics.NDCG()
            ),
            eval_method=base_method,
            n_trails=100
        )
        
    elif model_name == 'BPR':
        
        # define model
        model = BPR(max_iter=200, verbose=VERBOSE, seed=SEED)
        
        # define parameters to perform RandomSearch on
        bpr_params = [
            Discrete('k', list(range(5,101))),
            Continuous('lambda_reg', low=0.01, high=0.1),
            Continuous('learning_rate', low=0.001, high=0.01)
        ]

        
        rs_model = NewRandomSearch(
            model=model,
            space=bpr_params,
            metric=HarmonicMean(
                10,
                Serendipity(),
                cornac.metrics.FMeasure(k=10),
                cornac.metrics.NCRR(),
                cornac.metrics.NDCG()
            ),
            eval_method=base_method,
            n_trails=100
        )
  
    elif model_name == 'VAECF':
        
        # define model
        model = VAECF(autoencoder_structure = [30], act_fn='relu', likelihood='bern', verbose=VERBOSE, seed=SEED)

        # define parameters to perform RandomSearch on
        vaecf_params = [
            Discrete('k', list(range(5,101))),
            Discrete('n_epochs', list(range(5,51))),
            Discrete('batch_size', [8, 16, 32, 64]),
            Continuous('learning_rate', low=0.001, high=0.01)
        ]

        rs_model = NewRandomSearch(
            model=model,
            space=vaecf_params,
            metric=HarmonicMean(
                10,
                Serendipity(),
                cornac.metrics.FMeasure(k=10),
                cornac.metrics.NCRR(),
                cornac.metrics.NDCG()
            ),
            eval_method=base_method,
            n_trails=100
        )
        
    # run experiment
    experiment = cornac.Experiment(eval_method=base_method, models=[rs_model], metrics=eval_metrics)
    experiment.run()
    
    if model_name != 'MostPop':

        # print best params
        print('Random search best params: ', rs_model.best_params)

    EXPERIMENT = experiment

    # training set in dataframe
    TRAINING_SET = train

    # evaluate and calculate harmonic mean
    evaluate(EXPERIMENT, TRAINING_SET)

# 4. Model (likes)

## 4a. MostPop (Baseline)

In [7]:
model_name = 'MostPop'
weighted = 'No'
split = 'strat'
train_model(model_name, weighted, split)

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[MostPop] Training started!

[MostPop] Evaluation started!


Ranking:   0%|          | 0/100 [00:00<?, ?it/s]

Ranking:   0%|          | 0/96 [00:00<?, ?it/s]


VALIDATION:
...
        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------- + ------ + ------------ + ------- + ------- + ----------- + --------
MostPop | 0.0264 |       0.0070 |  0.0613 |  0.2528 |      0.0091 |   1.8221

TEST:
...
        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
MostPop | 0.0195 |       0.0067 |  0.0509 |  0.2378 |      0.0084 |    0.0000 |   2.1794

MostPop
F1@10: 0.020
NCRR: 0.051
NDCG: 0.238
Distributional coverage: 8.633
Serendipity: 0.814
Harmonic mean: 0.065



## 4b. WMF

In [8]:
model_name = 'WMF'
weighted = 'No'
split = 'strat'
train_model(model_name, weighted, split)

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_WMF] Training started!

[RandomSearch_WMF] Evaluation started!


Ranking:   0%|          | 0/100 [00:00<?, ?it/s]

Ranking:   0%|          | 0/96 [00:00<?, ?it/s]


VALIDATION:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_WMF | 0.0480 |       0.0110 |  0.0827 |  0.2690 |      0.0091 |   1.7920

TEST:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_WMF | 0.0250 |       0.0090 |  0.0488 |  0.2324 |      0.0084 |  235.7415 |   2.2274

Random search best params:  {'b': 0.025261213725309373, 'k': 8, 'lambda_u': 0.07386316292472243, 'lambda_v': 0.02037184386106846, 'learning_rate': 0.007595146617364617}
RandomSearch_WMF
F1@10: 0.025
NCRR: 0.049
NDCG: 0.232
Distributional coverage: 8.633
Serendipity: 0.814
Harmonic mean: 0.076



## 4c. BPR

In [9]:
model_name = 'BPR'
weighted = 'No'
split = 'strat'
train_model(model_name, weighted, split)

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_BPR] Training started!

[RandomSearch_BPR] Evaluation started!


Ranking:   0%|          | 0/100 [00:00<?, ?it/s]

Ranking:   0%|          | 0/96 [00:00<?, ?it/s]


VALIDATION:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_BPR | 0.0341 |       0.0081 |  0.0649 |  0.2570 |      0.0091 |   2.4646

TEST:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_BPR | 0.0237 |       0.0077 |  0.0571 |  0.2434 |      0.0084 |  229.8970 |   2.9866

Random search best params:  {'k': 90, 'lambda_reg': 0.02638440948158871, 'learning_rate': 0.0034852161694518707}
RandomSearch_BPR
F1@10: 0.024
NCRR: 0.057
NDCG: 0.243
Distributional coverage: 8.633
Serendipity: 0.814
Harmonic mean: 0.077



## 4d. VAECF

In [10]:
model_name = 'VAECF'
weighted = 'No'
split = 'strat'
train_model(model_name, weighted, split)

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_VAECF] Training started!

[RandomSearch_VAECF] Evaluation started!


Ranking:   0%|          | 0/100 [00:00<?, ?it/s]

Ranking:   0%|          | 0/96 [00:00<?, ?it/s]


VALIDATION:
...
                   |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------------------ + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_VAECF | 0.0311 |       0.0104 |  0.0737 |  0.2612 |      0.0091 |   1.8261

TEST:
...
                   |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------------------ + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_VAECF | 0.0439 |       0.0097 |  0.0698 |  0.2554 |      0.0084 |  195.3128 |   2.1799

Random search best params:  {'batch_size': 32, 'k': 61, 'learning_rate': 0.005950313122404854, 'n_epochs': 10}
RandomSearch_VAECF
F1@10: 0.044
NCRR: 0.070
NDCG: 0.255
Distributional coverage: 8.633
Serendipity: 0.814
Harmonic mean: 0.118



# 5. Model (weighted)

## 5a. WMF

In [11]:
model_name = 'WMF'
weighted = 'Yes'
split = 'strat'
train_model(model_name, weighted, split)

C:\Users\lim_j\AppData\Local\Temp\ipykernel_42588\713831474.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['log_dwell_time'] = train['dwell_time_ms'].apply(lambda x: np.log(x))


creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_WMF] Training started!

[RandomSearch_WMF] Evaluation started!


Ranking:   0%|          | 0/100 [00:00<?, ?it/s]

Ranking:   0%|          | 0/96 [00:00<?, ?it/s]


VALIDATION:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_WMF | 0.0314 |       0.0097 |  0.0687 |  0.2571 |      0.0091 |   1.8276

TEST:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_WMF | 0.0253 |       0.0094 |  0.0783 |  0.2552 |      0.0084 |  240.0232 |   2.2034

Random search best params:  {'b': 0.039183708492442515, 'k': 8, 'lambda_u': 0.02580558032415233, 'lambda_v': 0.09497617610357131, 'learning_rate': 0.005812383052067089}
RandomSearch_WMF
F1@10: 0.025
NCRR: 0.078
NDCG: 0.255
Distributional coverage: 8.633
Serendipity: 0.814
Harmonic mean: 0.087



## 5b. BPR

In [12]:
model_name = 'BPR'
weighted = 'Yes'
split = 'strat'
train_model(model_name, weighted, split)

C:\Users\lim_j\AppData\Local\Temp\ipykernel_42588\713831474.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['log_dwell_time'] = train['dwell_time_ms'].apply(lambda x: np.log(x))


creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_BPR] Training started!

[RandomSearch_BPR] Evaluation started!


Ranking:   0%|          | 0/100 [00:00<?, ?it/s]

Ranking:   0%|          | 0/96 [00:00<?, ?it/s]


VALIDATION:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_BPR | 0.0341 |       0.0081 |  0.0649 |  0.2570 |      0.0091 |   2.4711

TEST:
...
                 |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
---------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_BPR | 0.0237 |       0.0077 |  0.0571 |  0.2434 |      0.0084 |  232.0694 |   2.9961

Random search best params:  {'k': 90, 'lambda_reg': 0.02638440948158871, 'learning_rate': 0.0034852161694518707}
RandomSearch_BPR
F1@10: 0.024
NCRR: 0.057
NDCG: 0.243
Distributional coverage: 8.633
Serendipity: 0.814
Harmonic mean: 0.077



## 5c. VAECF

In [13]:
model_name = 'VAECF'
weighted = 'Yes'
split = 'strat'
train_model(model_name, weighted, split)

C:\Users\lim_j\AppData\Local\Temp\ipykernel_42588\713831474.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['log_dwell_time'] = train['dwell_time_ms'].apply(lambda x: np.log(x))


creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_VAECF] Training started!

[RandomSearch_VAECF] Evaluation started!


Ranking:   0%|          | 0/100 [00:00<?, ?it/s]

Ranking:   0%|          | 0/96 [00:00<?, ?it/s]


VALIDATION:
...
                   |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------------------ + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_VAECF | 0.0311 |       0.0104 |  0.0737 |  0.2612 |      0.0091 |   1.8546

TEST:
...
                   |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------------------ + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_VAECF | 0.0439 |       0.0097 |  0.0698 |  0.2554 |      0.0084 |  196.5974 |   2.1824

Random search best params:  {'batch_size': 32, 'k': 61, 'learning_rate': 0.005950313122404854, 'n_epochs': 10}
RandomSearch_VAECF
F1@10: 0.044
NCRR: 0.070
NDCG: 0.255
Distributional coverage: 8.633
Serendipity: 0.814
Harmonic mean: 0.118

